In [8]:
"""
fetch_macro.py

Fetches macroeconomic data (Interest Rate, CPI, Unemployment) from FRED API
and stores it in PostgreSQL.
"""

import os
import requests
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
FRED_API_KEY = os.getenv("FRED_API_KEY")

# Setup DB connection
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

# FRED series IDs
series = {
    "interest_rate": "FEDFUNDS",           # Federal Funds Rate
    "cpi": "CPIAUCNS",                     # Consumer Price Index
    "unemployment": "UNRATE"              # Unemployment Rate
}

def fetch_fred_series(series_id):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={FRED_API_KEY}&file_type=json"
    response = requests.get(url)
    data = response.json().get("observations", [])
    df = pd.DataFrame(data)[["date", "value"]]
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df["date"] = pd.to_datetime(df["date"])
    return df.set_index("date")

# Fetch and combine
macro_data = pd.DataFrame()
for name, series_id in series.items():
    df = fetch_fred_series(series_id)
    df.rename(columns={"value": name}, inplace=True)
    macro_data = pd.concat([macro_data, df], axis=1)

macro_data.reset_index(inplace=True)
macro_data["fetched_at"] = datetime.now()

# Store in PostgreSQL
macro_data.to_sql("macro_data", engine, if_exists="replace", index=False)
print("✅ Macro data saved to 'macro_data' table in PostgreSQL.")


✅ Macro data saved to 'macro_data' table in PostgreSQL.
